In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pymysql
from random import randint
import numpy as np
import time

USER_AGENTS = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
]


def get_page_html(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        return None

def get_article(bookurl):
    try:
        html = get_page_html(bookurl)
        soup = bs(html, "html.parser")
        bookintrobox = soup.find('span', attrs={'class': 'all hidden'})
        if bookintrobox:
            bookintro = bookintrobox.find('div',{'class':'intro'})
            bookintro = bookintro.get_text().replace('简介', '').strip()
        else:
            bookintro = soup.find('div',{'class':'intro'})
            bookintro = bookintro.get_text().replace('简介', '').strip()
        return bookintro
    except:
        return "0"

def saveinfo(title,bookurl,score,score_people,price,publishtime,publishcompany,author,comment,bookintro):
    db = pymysql.connect(host="localhost",user="zouzh",passwd="zouzihan0706",db="class")
    cursor = db.cursor()
    try:
#         print("db-connect")
        cursor.execute("INSERT INTO douban(title,bookurl,score,score_people,price,publishtime,publishcompany,author,comment,bookintro)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(title,bookurl,score,score_people,price,publishtime,publishcompany,author,comment,bookintro))
        db.commit()
#         print("db-ok")
    except Exception as e:
        print(e)
        db.rollback()
    db.close()

if __name__ == "__main__":
    urlbegin = "https://book.douban.com/top250?start="
    try:
        k = 0
        i=0
        while i < 250:
            url = urlbegin+str(i)
            print(url)

            random_agent = USER_AGENTS[randint(0, len(USER_AGENTS)-1)]
            headers = {
                'Connection':'close',
                'User-Agent':random_agent,
            }

            r = requests.get(url,headers=headers)
    #         print(r)

            if r.status_code == 200:
                rt = r.content
            else:
                rt = None
                print("ip被封了！")
            
            if rt:
                rh = str(rt,"utf-8")
                soup = bs(rh,"html.parser")
                tit = soup.find_all("tr",{"class","item"})
                # print(tit[0])
                for t in tit:
                    td = t.find("div",{"class","pl2"})
                    title = td.find("a").get("title")
        #             print(title)

                    bookurl = td.find("a").get("href")
                #     print(bookurl)

                    score = t.find('span', attrs={'class': 'rating_nums'})
                    score = score.get_text()
                #     print(score)

                    score_people = t.find('span', attrs={'class': 'pl'})
                    score_people = score_people.get_text()
                    score_people = score_people.replace('(', '').replace(')', '').strip()
                    score_people = score_people[:-3]
                #     print(score_people)

                    try:
                        info = t.find('p', attrs={'class': 'pl'})
                        info = info.get_text()
                        split = info.split('/')
                        price = split[-1].replace("元","").replace("CNY","").strip()
                    #     print(price)
                        split.pop(-1)
                        publishtime = split[-1].strip()
                    #     print(publishtime)
                        split.pop(-1)
                        publishcompany = split[-1].strip()
                    #     print(publishcompany)
                        split.pop(-1)
                        author = "、".join(split).replace("著","").replace("等","").strip()
                    #     print(author)
                    except:
                        price = "0"
                        publishtime = "0"
                        publishcompany = "0"
                        author = "0"

                    try:
                        comment = t.find('span',{"class":"inq"}).get_text().strip()
                #     print(comment)
                    except:
                        comment = "None"

                    bookintro = get_article(bookurl)
                #     print(bookintro)

                    try:
                        saveinfo(title,bookurl,score,score_people,price,publishtime,publishcompany,author,comment,bookintro)
                    except:
                        print("丢失了",url)
                    k = k+1
                    print(k)
                    time.sleep(np.random.uniform(0.5,2))
            i += 25
        print("successful!")
    except Exception as e:
        print(e)

https://book.douban.com/top250?start=0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
https://book.douban.com/top250?start=25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
https://book.douban.com/top250?start=50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
https://book.douban.com/top250?start=75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
https://book.douban.com/top250?start=100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
https://book.douban.com/top250?start=125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
https://book.douban.com/top250?start=150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
https://book.douban.com/top250?start=175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
1